In [ ]:
from scipy.spatial.distance import euclidean
import json
import numpy as np
from fastdtw import fastdtw
import os

In [ ]:
class aling_videos:

    def __init__(self, path_bar_trayectory, input_path, output_path):
        self.path_bar_trayectory = path_bar_trayectory
        self.input_path = input_path
        self.output_path = output_path
        self.trayectories = {}
        self.get_all_trayectories()

    def normalizar_posiciones_y(self, posiciones_y):
        pos_min = min(posiciones_y)
        pos_max = max(posiciones_y)
        amplitud_vertical = pos_max - pos_min

        if amplitud_vertical == 0:
            factor_escala = 1
        else:
            factor_escala = 1 / amplitud_vertical

        posiciones_y_normalizadas = np.array([[(pos - pos_min) * factor_escala] for pos in posiciones_y])
        return posiciones_y_normalizadas

    def get_trayectory(self, data):
        vertical_trayectory = np.zeros(len(data))
        trayectoria_normalizada = []
        for cnt, _ in enumerate(data):
            if len(data[cnt][0]) > 0:
                x1,y1,x2,y2,_ = data[cnt][0][0]
                center_x, center_y = int(x1 + (x2-x1)/2) , int(y1 + (y2 - y1)/2)
                vertical_trayectory[cnt] = center_y
            else:
                indices = np.nonzero(vertical_trayectory)[0]
                if len(indices) > 0:
                    j = indices[indices < cnt][-1]
                    value = vertical_trayectory[j]
                    vertical_trayectory[cnt] = value
        
        trayectoria_normalizada = self.normalizar_posiciones_y(vertical_trayectory)
        return trayectoria_normalizada

    def video_alignment(self, video1_embeddings, video2_embeddings):
        distance, path = fastdtw(video1_embeddings, video2_embeddings, dist=euclidean)
        return path, distance

    def takeSecond(self, elem):
        return elem[1]
    
    def get_top_videos(self, name, trayectory):
        files = [f for f in os.listdir(self.input_path)]
        best = []
        for file in files:
            name_ = file.split(".")[0]
            if name_ == name: continue
            trayectory_ = self.trayectories[name_] 
            path, cost = self.video_alignment(trayectory,trayectory_)
            best.append(((name_,path), cost))
        best.sort(key=self.takeSecond)
        data = {name : best}
        json_string = json.dumps(data)
        with open(self.output_path+name+'-alinged.json', 'w') as outfile:
            outfile.write(json_string)

    def get_all_trayectories(self):
        files = [f for f in os.listdir(self.input_path)]
        for file in files:
            name_ = file.split(".")[0]
            path = os.path.join(self.path_bar_trayectory, name_+".json")
            f = open(path)
            data = json.load(f) 
            trayectory_ = self.get_trayectory(data)
            self.trayectories[name_] = trayectory_

    def process_videos(self, i = 0, max_iter = None):
        files = [f for f in os.listdir(self.input_path)]
        files_sorted = sorted(files)    

        for j, file in enumerate(files_sorted):
            if j<i:continue
            if max_iter != None and j>max_iter: break
            name = file.split(".")[0]
            print("El video {} {} ha sido procesado".format(name,j))
            trayectory = self.trayectories[name] 
            self.get_top_videos(name, trayectory)

In [ ]:
path_bar_trayectory = "./bar_trajectories_raw/"
input_path = "./videos/"
output_path = "./alinged_videos_OHP/"

aling_ = aling_videos(path_bar_trayectory, input_path, output_path)

In [ ]:
aling_.process_videos()